# Using different languages inside single notebook

### Access to other languages via magics (given that they're installed)
- %%python2
- %%python3
- %%ruby
- %%perl
- %%bash
- %%R



In [43]:
%%ruby
puts 'Hi, this is ruby.'

Hi, this is ruby.


In [56]:
%%perl
print "Hello, this is perl\n";

Hello, this is perl


In [55]:
%%bash
echo "Hullo, I'm bash"

Hullo, I'm bash


## Writing functions in other languages

* Python and numpy can be too slow for certain tasks
* One can compile external functions and write python wrappers...
* but it's even simpler to directly use Fortran or Cython, hiding away the wrapping! 

In [36]:
!pip install cython fortran-magic

### Fortran
- fortranmagic doesn't work on my laptop because of f2py fail...


In [2]:
%load_ext fortranmagic

In [3]:
%%fortran
subroutine my_function(x, y, z)
    real, intent(in) :: x(:), y(:)
    real, intent(out) :: z(size(x))
    ! using vector operations  
    z(:) = sin(x(:) + y(:))
end subroutine

RuntimeError: f2py failed, see output

### Cython

- optimising static compiler for both Python and the extended Cython language
- makes writing C extensions for Python easy
- a superset of Python that additionally supports calling C functions and declaring C types on variables and class attributes

In [70]:
%%cython?

In [2]:
%load_ext Cython

In [3]:
%%cython
#%%cython --compile-args=-fopenmp --link-args=-fopenmp
def primes(int kmax):  # The argument will be converted to int or raise a TypeError.
    cdef int n, k, i  # These variables are declared with C types.
    cdef int p[1000]  # Another C type
    result = []  # A Python type
    if kmax > 1000:
        kmax = 1000
    k = 0
    n = 2
    while k < kmax:
        i = 0
        while i < k and n % p[i] != 0:
            i = i + 1
        if i == k:
            p[k] = n
            k = k + 1
            result.append(n)
        n = n + 1
    return result

In [4]:
%%timeit
p = primes(100)
#print p[-1]

10000 loops, best of 3: 25 µs per loop


In [5]:
def primes2(kmax):  # The argument will be converted to int or raise a TypeError.
    p = []
    result = []  # A Python type
    if kmax > 1000:
        kmax = 1000
    k = 0
    n = 2
    while k < kmax:
        i = 0
        while i < k and n % p[i] != 0:
            i = i + 1
        if i == k:
            p.append(n)
            k = k + 1
            result.append(n)
        n = n + 1
    return result

In [6]:
%%timeit
p = primes2(100)
#print p

100 loops, best of 3: 2.11 ms per loop


### Example: the Mandelbrot fractal

In [64]:
import numpy as np

Initialize:

In [65]:
size = 200
iterations = 100

#### Pure python

In [94]:
def mandelbrot_python(m, size, iterations):
    for i in range(size):
        for j in range(size):
            c = -2 + 3./size*j + 1j*(1.5-3./size*i)
            z = 0
            for n in range(iterations):
                if np.abs(z) <= 10:
                    z = z*z + c
                    m[i, j] = n
                else:
                    break

In [95]:
%%timeit -n1 -r1 
m = np.zeros((size, size))
mandelbrot_python(m, size, iterations)

1 loop, best of 1: 2.27 s per loop


#### First cython attempt

First just add the cython magic

In [85]:
%%cython
import numpy as np

def mandelbrot_cython(m, size, iterations):
    for i in range(size):
        for j in range(size):
            c = -2 + 3./size*j + 1j*(1.5-3./size*i)
            z = 0
            for n in range(iterations):
                if np.abs(z) <= 10:
                    z = z*z + c
                    m[i, j] = n
                else:
                    break

In [90]:
%%timeit -n1 -r1 
m = np.zeros((size, size), dtype=np.int32)
mandelbrot_cython(m, size, iterations)

1 loop, best of 1: 1.7 s per loop


Insignificant speedup...

#### Second attempt

Now add type information, and use memory views for NumPy arrays

In [98]:
%%cython
import numpy as np

def mandelbrot_cython(int[:,::1] m, 
                      int size, 
                      int iterations):
    cdef int i, j, n
    cdef complex z, c
    for i in range(size):
        for j in range(size):
            c = -2 + 3./size*j + 1j*(1.5-3./size*i)
            z = 0
            for n in range(iterations):
                if z.real**2 + z.imag**2 <= 100:
                    z = z*z + c
                    m[i, j] = n
                else:
                    break

In [99]:
%%timeit -n1 -r1 m = np.zeros((size, size), dtype=np.int32)
mandelbrot_cython(m, size, iterations)

1 loop, best of 1: 5.33 ms per loop


Huge speedup!